Toric code was proposed by Kitaev in paper 
"[Fault-tolerant quantum computation by anyons](https://arxiv.org/abs/quant-ph/9707021)"

[Leggett's intro](https://courses.physics.illinois.edu/phys598PTD/fa2013/L26.pdf) to toric code